In [72]:
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import cv2
import random

from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow import Tensor
from tensorflow.keras.optimizers import Adam

In [66]:
%%time
AIimages = []
new_size = (500, 500)
AI_path = "D:\AI vs Real Classification\AiArtData\AiArtData"

for file in os.listdir(AI_path):
    if file.endswith(".png") or file.endswith(".jpg"):
        img = Image.open(os.path.join(AI_path, file)).convert('L').resize(new_size)
        npArray = np.array(img)
        AIimages.append(npArray)
        
RealImages = []
Real_path = "D:\AI vs Real Classification\RealArt\RealArt"

for file in os.listdir(Real_path):
    if file.endswith(".png") or file.endswith(".jpg"):
        img = Image.open(os.path.join(Real_path, file)).convert('L').resize(new_size)
        npArray = np.array(img)
        RealImages.append(npArray)
        
AILabel = []
RealLabel = []
for _ in range(len(AIimages)): AILabel.append(0)
for _ in range(len(RealImages)): RealLabel.append(1)

C:\Users\jonso\anaconda3\envs\tf-gpu\lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


CPU times: total: 9.67 s
Wall time: 12.3 s


In [71]:
x_train = np.array(AIimages + RealImages)
y_train = np.array(AILabel + RealLabel)
print(x_train.shape)
print(y_train.shape)

(913, 500, 500)
(913,)


In [81]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
unison_shuffled_copies(x_train, y_train);

In [56]:
def build_model(pretrained=None):
    model = keras.Sequential([
        keras.Input(shape=(500, 500, 3)),
        layers.Conv2D(64, (3,3), activation = 'relu'),
        layers.Conv2D(64, (3,3), activation = 'relu'),
        layers.Conv2D(64, (3,3), activation = 'relu'),
        layers.Conv2D(64, (3,3), activation = 'relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(), layers.Dropout(0.5),
        layers.Dense(2, activation='softmax'),
    ])
    
    return model
build_model()

def train(model = build_model(), sp = 'model.h5', epochs=10):
    batch_size = 100
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.fit(
        x_train, 
        y_train,
        batch_size = batch_size,
        epochs = epochs,
        validation_split = 0.1)
    model.evaluate(x_test, y2_test, verbose=1)
    model.save(sp)